In [1]:
#@title Imports { form-width: "20%" }

!pip install ipython-autotime
%load_ext autotime
import numpy as np
import urllib.request
import re
from collections import Counter, defaultdict, namedtuple
from itertools import combinations, chain, product, permutations
import hashlib
from heapq import heappop, heappush
from functools import lru_cache
import copy



def get_input(day):
  return urllib.request.urlopen(
      f'https://raw.githubusercontent.com/SantoSimone/Advent-of-Code/master/'
      f'2016/input_files/input{day}.txt'
      ).read().decode('utf-8')

def get_input_as_lines(day):
  return get_input(day).split('\n')[:-1]

def parse_ints(text):
  return [int(x) for x in re.findall(r'\d+', text)]


time: 5.6 ms (started: 2021-03-16 20:06:35 +00:00)


# Day 1


In [ ]:
#@title Part 1 { form-width: "20%" }
def no_time_for_taxicab(input):
  # Parsing inputs
  moves = [(el[0], el[1:]) for el in [split.replace(' ', '') 
            for split in input.split(sep=',')]] # (dir, num)

  # Starting pos is center of plane heading North
  pos = 0
  head = 1j
  visited = [pos]

  for direction, steps in moves:    
    # turn right is a 90° clockwise, while left is 90° anti-clockwise
    head *= -1j if direction == 'R' else 1j
    for i in range(int(steps)):
      pos += 1 * head
      visited.append(pos)

  return abs(pos.real) + abs(pos.imag)


# It's day 1
day = 1

input = get_input(day)

no_time_for_taxicab(input)


In [ ]:
#@title Part 2 { form-width: "20%" }
# We just need to add a check in the function

def no_time_for_taxicab_pt2(input):
  # Parsing inputs
  moves = [(el[0], el[1:]) for el in [split.replace(' ', '') 
            for split in input.split(sep=',')]] # (dir, num)

  # Starting pos is center of plane heading North
  pos = 0
  head = 1j
  visited = [pos]

  for direction, steps in moves:    
    # turn right is a 90° clockwise, while left is 90° anti-clockwise
    head *= -1j if direction == 'R' else 1j
    for i in range(int(steps)):
      pos += 1 * head
      if pos in visited:
        return abs(pos.real) + abs(pos.imag)
      visited.append(pos)

  return abs(pos.real) + abs(pos.imag)


# It's day 1
no_time_for_taxicab_pt2(input)

# Day 2


In [ ]:
#@title Part 1 { form-width: "20%" }
# It's day 2
day = 2

# Split file into lines
lines = get_input_as_lines(day)

keypad = np.array([
  ['1', '2', '3'],
  ['4', '5', '6'],
  ['7', '8', '9']
])

# Encoding movements (Row, Col)
moves = {
  'U' : np.array([-1, 0]),
  'D' : np.array([+1, 0]),
  'L' : np.array([0, -1]),
  'R' : np.array([0, +1])
}
pos = np.array([1, 1])
code = ''

for line in lines:
  for move in line:
    pos += moves[move]
    pos = np.clip(pos, 0, keypad.shape[0] -1)    
  
  # End of line -> store actual value
  code += keypad[pos[0], pos[1]]

# Final code to bathroom
print(code)

In [ ]:
#@title Part 2 { form-width: "20%" }
# New keypad
keypad = np.array([
  ['0', '0', '1', '0', '0'],
  ['0', '2', '3', '4', '0'],
  ['5', '6', '7', '8', '9'],
  ['0', 'A', 'B', 'C', '0'],
  ['0', '0', 'D', '0', '0'],
])

# New rules
def should_i_stay_or_should_i_go(pos, move):
  move = pos + move
  move = np.clip(move, 0, 4)
  
  return pos if keypad[move[0], move[1]] == '0' else move

pos = np.array([2, 0])
code = ''

for line in lines:
  for move in line:
    pos = should_i_stay_or_should_i_go(pos, moves[move])
  
  code += keypad[pos[0], pos[1]]

  
print(code)


# Day 3

In [ ]:
#@title Part 1 { form-width: "20%" }

def check_triangle(tr):
  # Returns True if valid False otherwise
  x, y, z = sorted(tr)
  return z < x + y


# It's day 3
day = 3

# Split file into lines
lines = get_input_as_lines(day)
# Each line is a triangle
step = [line.split(' ') for line in lines]
# Filtering out empty splits
triangles = [parse_ints(l) for l in lines]

sum(map(check_triangle, triangles))

In [ ]:
#@title Part 2 { form-width: "20%" }

new_triangles = [
                 tr 
                 for i in range(0, len(triangles), 3)
                 for tr in zip(triangles[i], triangles[i+1], triangles[i+2])
                 ]

sum(map(check_triangle, new_triangles))

# Day 4

In [ ]:
#@title Part 1 { form-width: "20%" }

def parse_room(room):
  # Parse chars (with dashes), sector id and checksum for each room entry
  return re.match(r'([a-z-]+)-(\d+)\[([a-z]+)\]', room).groups()

def check_room(room):
  chars, id, checksum = parse_room(room)
  # collections.Counter obj performs exactly what we want
  counts = Counter(chars.replace('-', ''))
  # Getting real checksum
  real = ''.join(
      [v[0] for v in sorted(counts.items(), key=lambda x: (-x[1], x[0]))][:5]
      )
  return int(id) if real == checksum else 0


# It's day 4
day = 4

# Split file into lines
lines = get_input_as_lines(day)
# Each line is a room
rooms = [line for line in lines]

sum = np.sum([check_room(r) for r in rooms])
sum

In [ ]:
#@title Part 2 { form-width: "20%" }

def decrypt(room):
  chars, id, checksum = parse_room(room)
  # First, we decrypt a single word, then we concatenates with whitespaces
  ret = ' '.join([ 
                ''.join([chr(97 + (ord(ch) - 97 + int(id)) % 26)
                for ch in word])
        for word in chars.split('-') 
        ])
  return ret, int(id)

for room in rooms:
  msg, id = decrypt(room)
  if 'north' in msg:
    print(id)


# Day 5

In [ ]:
#@title Part 1 { form-width: "20%" }
# It's day 5
day = 5

# my input
input = 'ojvtpuvg'

# PART ONE
i = 0
pw = ''
while True:
  ex = f'{input}{i}'
  code = hashlib.md5(ex.encode()).hexdigest()
  if code.startswith('00000'):
    pw += code[5]
    if len(pw) == 8: break

  i += 1

print(f'Final password: {pw}')

In [ ]:
#@title Part 2 { form-width: "20%" }
# It takes longer, so we do some random stuff to make it fancier
input = 'ojvtpuvg'
i = 0
pw = ['_'] * 8
changed = 0
while True:
  ex = f'{input}{i}'
  code = hashlib.md5(ex.encode()).hexdigest()
  if code.startswith('00000') and code[5].isdigit() and int(code[5]) < 8 and pw[int(code[5])] == '_':    
    changed += 1
    pw[int(code[5])] = code[6]
    print(f'\rDecrypted password: {"".join(pw)}', end='')
    if changed == 8: break
  i += 1

# Day 6


In [ ]:
#@title Part 1 { form-width: "20%" }
# It's day 6
day = 6

lines = get_input_as_lines(day)

positions = zip(*lines)
counters = [Counter(pos) for pos in positions]

# getting top 1 occurence
msg = ''.join([count.most_common(1)[0][0] for count in counters])

print(msg)

In [ ]:
#@title Part 2 { form-width: "20%" }
# Difference in p2 is that we take the least common

# getting all occurences sorted, then getting the last
msg = ''.join([count.most_common()[-1][0] for count in counters])
print(msg)

# Day 7

In [ ]:
#@title Part 1 { form-width: "20%" }

def check_abba(word):
    splits = [word[i:i + 4] for i in range(len(word) - 3)]
    return any([s[:2] == ''.join(reversed(s[2:])) and s[0] != s[1] 
                for s in splits])


def support_TLS(line):
    splits = re.split(r'\[|\]', line)
    hypernets = splits[1::2]
    supernets = splits[0::2]

    return not any(map(check_abba, hypernets)) \
           and any(map(check_abba, supernets))

# It's day 7
day = 7
lines = get_input_as_lines(day)
s = np.sum([support_TLS(line) for line in lines])
print(f'TLS compliant IPs: {s}')

In [ ]:
#@title Part 2 { form-width: "20%" }

def check_aba(word, hypernets):
    splits = [word[i:i + 3] for i in range(len(word) - 2)]
    return any([s[0] == s[2] != s[1] and any([check_bab(''.join([s[1], s[0], s[1]]), br) for br in hypernets])
                for s in splits])

def check_bab(check, word):
    return check in word

def support_SSL(line):
    splits = re.split(r'\[|\]', line)
    hypernets = splits[1::2]
    supernets = splits[0::2]

    return any([check_aba(s, hypernets) for s in supernets])


lines = get_input_as_lines(day)
s = np.sum([support_SSL(line) for line in lines])
print(f'SSL compliant IPs: {s}')

# Day 8

In [ ]:
#@title Part 1 { form-width: "20%" }

def exec(line, screen):
  if re.findall('rect', line):
      w, h = parse_ints(line)
      screen[:h, :w] = 1
  elif re.findall('rotate column', line):
      col, shift = parse_ints(line)
      screen[:, col] = np.roll(screen[:, col], shift, axis=0)
  elif re.findall('rotate row', line):
      row, shift = parse_ints(line)
      screen[row, :] = np.roll(screen[row, :], shift, axis=0)

  return screen

# It's day 8
day = 8
lines = get_input_as_lines(day)
screen = np.zeros((6, 50), dtype=np.int32)
for line in lines:
    screen = exec(line, screen)
print(f'Final number of lights: {np.sum(screen, (0,1))}')

In [ ]:
#@title Part 2 { form-width: "20%" }
def print_screen(screen):
  for i in range(screen.shape[0]):
    for j in range(screen.shape[1]):
      if screen[i, j]:
        print('#', end='')
      else:
        print(' ', end='')
    print(' ')

print('Final screen: ')
print_screen(screen)

# Day 9

In [ ]:
#@title Part 1 { form-width: "20%" }

def next_marker(txt):
  search = re.search(r'\((\d+)x(\d+)\)|$', txt)
  start, stop = search.span()
  if start == stop: return start, stop, 0, 0
  length, reps = map(int, search.groups())
  return start, stop, length, reps


def calc_len(txt):
  total = 0
  while True:
      start, stop, length, reps = next_marker(txt)
      total += start + length * reps
      i = stop + length
      if i >= len(txt): break
      txt = txt[i:]
  return total

# It's day 9
day = 9
txt = ''.join(get_input_as_lines(day))
length = calc_len(txt)
print(f'Total length: {length}')

In [ ]:
#@title Part 2 { form-width: "20%" }

# We need to add a bit of recursion here
def calc_len_v2(txt):
  total = 0
  while True:
      start, stop, length, reps = next_marker(txt)
      if start == stop: return start
      total += start + calc_len_v2(txt[stop:length + stop]) * reps
      i = stop + length
      if i >= len(txt): break
      txt = txt[i:]
  return total

length = calc_len_v2(txt)
print(f'Total length: {length}')

# Day 10

In [ ]:
#@title Part 1 { form-width: "20%" }

def d10p1(txt):
  def pass_to(src, dst, chip):
    if src != 'input':
      owns[src].discard(chip)
    owns[dst].add(chip)
    current = owns[dst]
    if current == finish:
      print(f'The bot we want is {dst}')
    if len(current) == 2:
      pass_to(dst, rules[dst][0], min(current))
      pass_to(dst, rules[dst][1], max(current))

  owns = defaultdict(set)
  rules = {
      owner: (dst1, dst2)
      for (owner, dst1, dst2) in 
      re.findall(r'(bot \d+) gives low to (\w+ \d+) and high to (\w+ \d+)', txt)
  }
  finish = {17, 61}
  for chip, dst in re.findall(r'value (\d+) goes to (\w+ \d+)', txt):
    pass_to('input', dst, int(chip))
  
  return owns

day = 10
txt = get_input(day)
owns = d10p1(txt)

The bot we want is bot 93


In [ ]:
#@title Part 2 { form-width: "20%" }

out0 = owns['output 0'].pop()
out1 = owns['output 1'].pop()
out2 = owns['output 2'].pop()

print(f'Result of mul of first 3 outputs is {out0 * out1 * out2}')

Result of mul of first 3 outputs is 47101


# Day 11

In [ ]:
#@title Part 1 { form-width: "20%" }

def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return step(prev_states, state)
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs or new_cost < costs[new_state]:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def step(prev, state):
  return ([] if state is None else step(prev, prev[state]) + [state])

def heuristic(state):
  # Calculating how many more moves we need to finish (heuristic obviously)
  # The idea is that we can move 2 items at the same time and we count how many
  # floors needs every item to reach top floor
  remaining = sum(len(floor) * i for (i, floor) in enumerate(reversed(state.floors)))
  return np.ceil(remaining / 2)

def moves(state):
  # Possible moves from actual state
  lvl, floors = state

  # Elevator can move 1 level at a time
  for new_lvl in {min(lvl + 1, 3), max(lvl - 1, 0)}:
    # We can bring two things at most
    for stuff in obj_to_move(floors[lvl]):
      new_floors = tuple(
          (s | stuff if i == new_lvl else # Adding new stuff to the level we are heading 
           s - stuff if i == state.elevator else # Removing stuff taken from this level
           s) # Unchanged level cause we are moving in other floors
           for (i, s) in enumerate(state.floors)
           )
      # Check if the stuff we are moving isn't destroying the building
      if legal_floor(new_floors[lvl]) and legal_floor(new_floors[new_lvl]):
        yield State(new_lvl, new_floors)

def legal_floor(floor):
  # Checking if the floor isn't going to implode
  # Generators in this floor
  gens = any(g.endswith('G') for g in floor)
  # Chips in this floor
  chips = [c for c in floor if c.endswith('M')]
  # A floor is legal if there's no generator on it
  # or if every chip is connected to its generator
  return not gens or all(c[0] + 'G' in floor for c in chips)


def obj_to_move(objs):
  for x in chain(combinations(objs, 1), combinations(objs, 2)):
    yield frozenset(x)

def fs(*items): return frozenset(items)


State = namedtuple('State', 'elevator, floors')
floors = {0, 1, 2, 3}
day = 11
start = State(0, (fs('PG', 'TG', 'TM', 'pG', 'RG', 'RM', 'CG', 'CM'),
                  fs('PM', 'pM'),
                  fs(),
                  fs()))
path = problem_solver(start, heuristic, moves)
print(f'Response is the length of the path to the top: {len(path) -1}')

Response is the length of the path to the top: 47


In [ ]:
#@title Part 2 { form-width: "20%" }
# Slow timing, should implement a better heuristic to avoid 'full' grid search

start  = State(0, (fs('PG', 'TG', 'TM', 'pG', 'RG', 'RM', 'CG', 'CM', 'EG', 'EM', 'DG', 'DM'),
                  fs('PM', 'pM'),
                  fs(),
                  fs()))

path = problem_solver(start, heuristic, moves)
print(f'Response is the length of the path to the top: {len(path) -1}')

# Day 12

In [ ]:
#@title Part 1 { form-width: "20%" }

def d12p1(instructions, regs):
  # Decrypting password from instructions
  i = 0
  while i < len(instructions):
    splits = instructions[i].split(' ')
    i += execute(splits, regs)
    
  return regs

def execute(ex, regs):
  # Execution of single instruction -> Returning the offset for next instruction
  ret = 1
  if ex[0] == 'cpy':
    regs[ex[2]] = regs[ex[1]] if ex[1].isalpha() else int(ex[1])
  elif ex[0] == 'inc':
    regs[ex[1]] += 1
  elif ex[0] == 'dec':
    regs[ex[1]] -= 1
  elif ex[0] == 'jnz':
    check = regs[ex[1]] if ex[1].isalpha() else ex[1]
    ret = int(ex[2]) if check != 0 else 1
  return ret

# It's day 12
day = 12
lines = get_input_as_lines(day)
regs = {
    'a': 0,
    'b': 0,
    'c': 0,
    'd': 0
}
ret = d12p1(lines, regs)['a']
print(f'Reg \'a\' contains: {ret}')

Registry a contains: 318009


In [ ]:
#@title Part 2 { form-width: "20%" }

# Just need to wait a bit longer
regs = {
    'a': 0,
    'b': 0,
    'c': 1,
    'd': 0
}
ret = d12p1(lines, regs)['a']

print(f'Reg \'a\' contains: {ret}')

Reg 'a' contains: 9227663


# Day 13

In [ ]:
#@title Part 1 { form-width: "20%" }

# Optimization problem means problem_solver being helpful
def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return step(prev_states, state)
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs or new_cost < costs[new_state]:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def heuristic(state):
  # Our heuristic is the city-distance between this state and goal state
  return abs(state.x - goal['x']) + abs(state.y - goal['y']) 

def moves(state):
  # Possible moves from this state
  x, y = state
  for x2, y2 in list(product([x], [y-1, y+1])) + list(product([x-1, x+1], [y])):
    if legal_move(x2, y2):
      yield State(x2, y2)

def legal_move(x, y):
  if x < 0 or y < 0:
    return False

  res = x*x + 3*x + 2*x*y + y + y*y + favorite
  return False if count_ones_binary(res) % 2 else True

def step(prev, state):
  return ([] if state is None else step(prev, prev[state]) + [state])

def count_ones_binary(num):
  return sum([1 if x == '1' else 0 for x in list(f'{num:b}')])

State = namedtuple('State', 'x, y')

# It's day 13, but we have no input to get from the web
goal = {'x': 31, 'y': 39}
favorite = 1362
path = problem_solver(State(1,1), heuristic, moves)
print(f'Number of moves to reach (31,39) is: {len(path)}')

Number of moves to reach (31,39) is: 83


In [ ]:
#@title Part 2 { form-width: "20%" }

# We need a new func
def distinct_locations(start, N, f_moves):
    # Heap with next state to be evaluated
  heap = [start]
  # Dict to keep track num of locations
  distance = {start: 0}

  while heap:
    (state) = heappop(heap)
    # Save the location if we did less than 50 steps
    if distance[state] < N:
      for new_state in f_moves(state):
        if new_state not in distance:
          # Never seen state OR better solution than previous one
          heappush(heap, new_state)
          distance[new_state] = distance[state] +1

  return len(distance)

# Actual Run
locs = distinct_locations(State(1,1), 50, moves)
print(f'Max distinct locations visited in 50 moves: {locs}')

Max distinct locations visited in 50 moves: 138


# Day 14

In [ ]:
#@title Part 1 { form-width: "20%" }

@lru_cache(1001)
def hash_gen(code):
  code = hashlib.md5(code.encode()).hexdigest()
  return code

def is_key(salt, i):
  # First rule: same char repeating 3 times in MD5 hash
  match = re.search(r'(.)\1\1', hash_gen(f'{salt}{i}'))
  if match:
    ch = match.group(1) * 5
    # Second rule: one of the next 1000 MD5 hashes must contain a char 
    # repeating 5 times
    return 1 if any(ch in hash_gen(f'{salt}{off}')
                    for off in range(i + 1 , i + 1001)) else 0
    
  return 0

def d14p1(salt):
  i = 0
  keys = 0
  while keys < 64:
    keys += is_key(salt, i)
    i += 1
  
  return i - 1

salt = 'cuanljph'
print(f'64-th key is generated by index: {d14p1(salt)}')

64-th key is generated by index: 23769


In [ ]:
#@title Part 2 { form-width: "20%" }

@lru_cache(1001)
def hash_gen(code):
  for j in range(2017):
    code = hashlib.md5(code.encode()).hexdigest()
  return code

print(f'64-th key is generated by index: {d14p1(salt)}')

64-th key is generated by index: 20606


# Day 15

In [ ]:
#@title Part 1 { form-width: "20%" }

def starting_discs(txt):
  # Each disc has a pos and a total number of positions
  return [(int(x[0]), int(x[1]), int(x[2]))
          for x in re.findall(r'#(\d+) .+ (\d+) positions.+position (\d+)',txt)]

def button_pushed(time, discs):
  for pos, tot, start in discs:
    if ((start + pos + time) % tot) != 0:
      return False
  return True

def d15p1(txt):
  time = 0
  ret = False

  while not ret:
    discs = starting_discs(txt)
    ret = button_pushed(time, discs) 
    time += 1

  # We updated time even if we found our solution -> time - 1
  return time - 1

# It's day 15
day = 15
txt = get_input(day)
ret = d15p1(txt)
print(f'I got my capsule at time: {ret}')

I got my capsule at time: 121834


In [ ]:
#@title Part 2 { form-width: "20%" }

# I will add the new input hard-coded as it is faster
def d15p2(txt):
  time = 0
  ret = False

  while not ret:
    discs = starting_discs(txt)
    discs.append((7, 11, 0)) # Here's the change
    ret = button_pushed(time, discs) 
    time += 1

  # We updated time even if we found our solution -> time - 1
  return time - 1

# It's day 15
day = 15
txt = get_input(day)
ret = d15p2(txt)
print(f'I got my capsule at time: {ret}')

I got my capsule at time: 3208099


# Day 16

In [ ]:
#@title Part 1 { form-width: "20%" }

def produce_data(data, L):
  while len(data) < L:
    b = data[:]
    b = b[::-1]
    b = b.replace('0', '_').replace('1', '0').replace('_', '1')
    data += '0' + b
  return data

def checksum(data):
  return ''.join(
      ['1' if data[i] == data[i+1] else '0' for i in range(0, len(data), 2)]
  )

def d16p1(start, L):
  data = produce_data(start, L)
  check = checksum(data[:L])
  while (len(check) % 2) == 0:
    check = checksum(check)

  return check

# My inputs
start = '01110110101001000'
L = 272
check = d16p1(start, L)
print(f'Checksum of my input is: {check}')

Checksum of my input is: 11100111011101111


In [ ]:
#@title Part 2 { form-width: "20%" }

L = 35651584
check = d16p1(start, L)
print(f'Checksum of my input is: {check}')

Checksum of my input is: 10001110010000110


# Day 17

In [ ]:
#@title Part 1 { form-width: "20%" }

# Once again, we need a problem solver
def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return state.path
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def heuristic(state):
  # Our heuristic will be the distance between here and goal
  return abs(state.x - goal['x']) + abs(state.y - goal['y'])

def moves(state):
  x, y, path = state
  legal_moves = legal_move(path)
  if legal_moves[0] and 0 <= y - 1 <= 3: yield State(x, y - 1, path + 'U')
  if legal_moves[1] and 0 <= y + 1 <= 3: yield State(x, y + 1, path + 'D')
  if legal_moves[2] and 0 <= x - 1 <= 3: yield State(x - 1, y, path + 'L')
  if legal_moves[3] and 0 <= x + 1 <= 3: yield State(x + 1, y, path + 'R')

  
def legal_move(path):
  # Checkin hashing
  code = passcode + path
  h = hashlib.md5(code.encode()).hexdigest()[:4]
  return [True if x in 'bcdef' else False for x in h]

State = namedtuple('State', 'x, y, path')

# My input
passcode = 'bwnlcvfs'
goal = {'x': 3, 'y': 3}
path = problem_solver(State(0, 0, ''), heuristic, moves)
print(f'Fastest path is {path}')

Fastest path is DDURRLRRDD


In [ ]:
#@title Part 2 { form-width: "20%" }

# Our problem solver needs to change its habits
def new_solver(start, f_moves):
  # Heap with next state to be evaluated
  heap = [start]
  ret = 0

  while heap:
    state = heap.pop()
    if state.x == 3 and state.y == 3:
      # Goal state, we're done
      ret = max(ret, len(state.path))
    else:
      heap.extend(f_moves(state))
  
  return ret

path = new_solver(State(0, 0, ''), moves)
print(f'Longest path is {path}')

Longest path is 436


# Day 18

In [ ]:
#@title Part 1 { form-width: "20%" }

def safe_or_not(pos, prev):
  # Check if i'm going to die
  left = prev[pos - 1] if pos - 1 >= 0 else '.'
  center = prev[pos]
  right = prev[pos + 1] if pos + 1 < len(prev) else '.'
  if left == center == '^' and right == '.': return '^'
  if center == right == '^' and left == '.': return '^'
  if center == right == '.' and left == '^': return '^'
  if center == left == '.' and right == '^': return '^'
  
  return '.'

def d18p1(start, N):
  row = start
  ret = [start]
  safe = sum([1 if x == '.' else 0 for x in start])
  for i in range(0, N - 1):
    row = ''.join([safe_or_not(pos, row) for pos, _ in enumerate(row)])
    safe += sum([1 if x == '.' else 0 for x in row])
    ret.append(row)

  return ret, safe

# It's day 18
day = 18
txt = get_input(day).replace('\n', '')
N = 40
map, safe = d18p1(txt, N)
print(f'In first {N} rows we have {safe} safe tiles!')

In first 40 rows we have 1974 safe tiles!
time: 197 ms (started: 2021-03-14 08:31:16 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

# Just a bit longer waiting
N = 400000
map, safe = compute_tiles(txt, N)
print(f'In first {N} rows we have {safe} safe tiles!')

In first 400000 rows we have 19991126 safe tiles!
time: 24.9 s (started: 2021-03-14 08:31:19 +00:00)


# Day 19

In [ ]:
#@title Part 1 { form-width: "20%" }

# Super short version, so we need to document a bit
# Credits to @norvig for intuition

def winner(elves):
  # If we have one element, he's the winner, otherwise we increase the distance
  # between 2 elves with presents (increasing distance means we double the
  # range)
  return (elves[0] if (len(elves) == 1) else winner(one_round(elves)))

def one_round(elves): 
  # If number of elves is even we skip odd numbers (elf 1-3-5.. get their
  # presents stolen)
  # If the number is odd we have the same pattern, beside last one that steals
  # from first
  return (elves[0::2] if (len(elves) % 2 == 0) else elves[2::2])

elves = 3014603
ret = winner(range(1, elves + 1))
print(f'The winner is number {ret}')

The winner is number 1834903
time: 7.31 ms (started: 2021-03-14 08:35:38 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

# We need to redefine the round
def one_round(elves):
  N = len(elves)
  # We need to skip the ones eliminated so we keep track of how many they are
  eliminated = 0
  for i in range(int(np.ceil(N / 3))):
    across = i + eliminated + (N // 2)
    elves[across] = None
    N -= 1
    eliminated += 1
  return list(filter(None, elves[i+1:] + elves[:i+1]))

elves = 3014603
winner(list(range(1, elves + 1)))

1420280

time: 1.3 s (started: 2021-03-14 08:36:08 +00:00)


# Day 20

In [ ]:
#@title Part 1 { form-width: "20%" }

# We use too much RAM if we try to build the blocked numbers, so we do it
# the other way
# We yield the range of values between current IP and minimum value of the
# current range of blocked
# We update the current candidate IP with the maximum value
def find_lowest(blacklisted):
  ip = 0
  for min_r, max_r in blacklisted:
    if len(range(ip, min_r)) > 0:
      yield range(ip, min_r) 
    ip = max(ip, max_r + 1)
  
def d20p1(txt):
  blacklisted = sorted(
      [(int(x), int(y))
       for x, y in re.findall(r'(\d+)-(\d+)', txt)],
      key=lambda x: x[0]
  )

  return list(find_lowest(blacklisted))

# It's day 20
day = 20
txt = get_input(20)
ret = d20p1(txt)
print(f'Lowest IP is {list(ret[0])[0]}')

Lowest IP is 17348574
time: 59.9 ms (started: 2021-03-14 08:52:36 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

count = len([list(r) for r in ret])
print(f'Total number of IPs allowed is {count}')

Total number of IPs allowed is 104
time: 5.75 ms (started: 2021-03-14 08:52:36 +00:00)


# Day 21

In [ ]:
#@title Part 1 { form-width: "20%" }

def rotate(pw, s, dir):
  return pw[-s:] + pw[:-s] if dir == 'right' else pw[s:] + pw[:s]

def apply_move(line, pw):
  if line.startswith('swap position'):
    x, y = re.findall(r'(\d) with position (\d)', line)[0]
    pw[int(x)], pw[int(y)] = pw[int(y)], pw[int(x)]
  elif line.startswith('swap letter'):
    x, y = re.findall(r'([a-z]) with letter ([a-z])', line)[0]
    idx_x, idx_y = pw.index(x), pw.index(y)
    pw[idx_x], pw[idx_y] = pw[idx_y], pw[idx_x]
  elif re.match(r'rotate [left|right]', line) is not None:
    pw = rotate(pw, int(re.findall(r'(\d)', line)[0]), re.findall(r'(left|right)', line)[0])
  elif line.startswith('rotate based'):
    x = re.findall(r'letter ([a-z])', line)[0]
    idx = pw.index(x)
    pw = rotate(pw, 1, 'right')
    pw = rotate(pw, idx, 'right')
    if idx >= 4:
      pw = rotate(pw, 1, 'right')
  elif line.startswith('reverse'):
    x, y = re.findall(r'(\d+) through (\d+)', line)[0]
    x, y = int(x), int(y)
    pw[x:y+1] = pw[x:y+1][::-1]
  elif line.startswith('move'):
    x, y = re.findall(r'(\d+) to position (\d+)', line)[0]
    x, y = int(x), int(y)
    pw[x:x+1], pw[y:y] = [], pw[x:x+1]
    
  return pw

def d21p1(lines, pw):
  for line in lines:
    pw = apply_move(line, pw)

  return ''.join(pw)

# It's day 21
day = 21
lines = get_input_as_lines(day)
start = list('abcdefgh')
ret = d21p1(lines, start)
print(f'Result of scrambling is {ret}')

Result of scrambling is hcdefbag
time: 220 ms (started: 2021-03-14 09:02:40 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

# Bit lazy, so i go with the brute force way
def d21p2(lines, goal):
  for p in permutations(goal):
    if scramble(lines, list(p)) == goal:
      return p

goal = 'fbgdceah'
ret = d21p2(lines, goal)
print(f'Unscrambling \'fbgdceah\' gives: {ret}')

# Day 22

In [3]:
#@title Part 1 { form-width: "20%" }

def parse_int(line):
  return [int(x) for x in re.findall(r'\d+', line)]

def viable_pair(a, b):
  return a != b and 0 < a.used <= b.avail

def d22p1(lines):
  grid = [Node(*parse_int(line)) for line in lines if line.startswith('/dev')]
  count = sum([viable_pair(a, b) for a, b in permutations(grid, 2)])

  return count

Node = namedtuple('Node', 'x, y, size, used, avail, perc')

# It's day 22
day = 22
lines = get_input_as_lines(day)
ret = d22p1(lines)
print(f'Viable pairs of my input are {ret}')

Viable pairs of my input are 1034
time: 463 ms (started: 2021-03-16 20:07:02 +00:00)


In [7]:
#@title Part 2 { form-width: "20%" }

# YAOP! (Yet Another Optimization Problem)
State = namedtuple('State', 'goalpos, emptypos')

def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return step(prev_states, state)
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs or new_cost < costs[new_state]:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def heuristic(state):
  # Our heuristic is the city-distance between datagoal position of this state and (0,0)
  return sum(state.goalpos)

def moves(state):
  # Possible moves from this state
  (x, y), (x_empty, y_empty) = state
  for x2, y2 in list(product([x_empty], [y_empty-1, y_empty+1])) + list(product([x_empty-1, x_empty+1], [y_empty])):
    if 0 <= x2 <= x_max and 0 <= y2 <= y_max and legal_move(grid[x2, y2], grid[empty]):
      newgoal = (x_empty, y_empty) if x2 == x and y2 == y else (x, y)
      yield State(newgoal, (x2, y2))

def emptypos(grid):
  for node in grid.items():
    if node[1].used == 0:
      return node[1].x, node[1].y

def legal_move(a, b):
  return True if a.used <= b.size else False

def step(prev, state):
  return 0 if state is None else 1 + step(prev, prev[state])

# It's day 22
day = 22
lines = get_input_as_lines(day)
grid = [Node(*parse_int(line)) for line in lines if line.startswith('/dev')]
grid = {(node.x, node.y): node for node in grid}
x_max = max(node[1].x for node in grid.items())
y_max = max(node[1].y for node in grid.items())
empty = emptypos(grid)
ret = problem_solver(State((x_max, 0), empty), heuristic, moves)
print(f'Fewest number of steps required is {ret - 1}')

Fewest number of steps required is 261
time: 10.4 s (started: 2021-03-16 20:12:06 +00:00)


# Day 23

In [13]:
#@title Part 1 { form-width: "20%" }

def d23p1(txt, regs):
  lines = [[x if x.isalpha() else int(x) 
            for x in line.split()] for line in txt.splitlines()]

  def exec_tgl(pos):
    if 0 <= pos < len(lines):  
      if lines[pos][0] == 'inc':
        lines[pos][0] = 'dec'
      elif len(lines[pos]) == 2:
        lines[pos][0] = 'inc'
      elif lines[pos][0] == 'jnz':
        lines[pos][0] = 'cpy'
      else:
        lines[pos][0] = 'jnz'

  i = 0
  while 0 <= i < len(lines):
    inst = lines[i]
    i += 1
    op, x, y = inst[0], inst[1], inst[-1]
    if op == 'cpy' and y in regs:
      regs[y] = regs[x] if x in regs else x
    elif op == 'inc':
      regs[x] += 1
    elif op == 'dec':
      regs[x] -= 1
    elif op == 'jnz':
      check = regs[x] if x in regs else x
      move = regs[y] if y in regs else y
      i += move - 1 if check != 0 else 0
    elif op == 'tgl':
      pos = regs[x] if x in regs else x
      exec_tgl(i + pos - 1)

  return regs

# It's day 23
day = 23
regs = {
    'a': 7,
    'b': 0,
    'c': 0,
    'd': 0
}
txt = get_input(day)
ret = d23p1(txt, regs)['a']
print(f'Registry \'a\' contains: {ret}')

Registry a contains: 11123
time: 115 ms (started: 2021-03-16 20:18:26 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }

# Solution is pretty slow, but I hate assembly coding and I don't want to
# reinvent multiplications :)
regs = {
    'a': 12,
    'b': 0,
    'c': 0,
    'd': 0
}
txt = get_input(day)
ret = d23p1(txt, regs)['a']
print(f'Registry \'a\' contains: {ret}')

# Day 24

In [19]:
#@title Part 1 { form-width: "20%" }
# Guess who's back
def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return step(prev_states, state)
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs or new_cost < costs[new_state]:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def step(prev, state):
  return ([] if state is None else step(prev, prev[state]) + [state])

def heuristic(state):
  # Calculating how many more moves we need to finish (heuristic obviously)
  return goal - len(state.collected)

def moves(state):
  # Possible moves from actual state
  x, y, collected = state
  
  for x2, y2 in neighbors(x, y):
    if legal_move(x2, y2):
      yield State(x2, y2, fs(*(chain(collected, lines[y2][x2])))) if lines[y2][x2].isdigit() and lines[y2][x2] not in collected else State(x2, y2, fs(*collected))

def neighbors(x, y):
  return (x+1, y), (x-1, y), (x, y+1), (x, y-1)

def legal_move(x, y):
  return 0 <= x < len(lines[0]) and 0 <= y < len(lines) and lines[y][x] != '#'

def obj_to_move(objs):
  for x in chain(combinations(objs, 1), combinations(objs, 2)):
    yield frozenset(x)

def get_start(lines):
  for y, line in enumerate(lines):
    for x, ch in enumerate(line):
      if ch == '0':
        return x, y

def fs(*items): return frozenset(items)

State = namedtuple('State', 'x, y, collected')

# It's day 24
day = 24
goal = sum(x.isdigit() for line in lines for x in line)
lines = get_input_as_lines(day)
start = get_start(lines)
ret = problem_solver(State(start[0], start[1], fs('0')), heuristic, moves)
print(f'Number of steps needed is {len(ret) -1}')

Number of steps needed is 464
time: 3.49 s (started: 2021-03-16 20:33:34 +00:00)


In [20]:
#@title Part 2 { form-width: "20%" }

# We need to redefine the heuristic to get into account to return to position 0
def heuristic(state):
  nums = goal - len(state.collected)
  dist = abs(state.x - start[0]) + abs(state.y - start[1])
  return nums + dist

# It's day 24
day = 24
goal = sum(x.isdigit() for line in lines for x in line)
lines = get_input_as_lines(day)
start = get_start(lines)
ret = problem_solver(State(start[0], start[1], fs('0')), heuristic, moves)
print(f'Number of steps needed is {len(ret) -1}')

Number of steps needed is 652
time: 7.49 s (started: 2021-03-16 20:33:50 +00:00)


# Day 25

In [24]:
#@title Part 1 { form-width: "20%" }

# Actually a day 23 remake
def decrypt(regs, lines):
  i = 0
  ret = ''

  def exec_tgl(pos):
    if 0 <= pos < len(lines):  
      if lines[pos][0] == 'inc':
        lines[pos][0] = 'dec'
      elif len(lines[pos]) == 2:
        lines[pos][0] = 'inc'
      elif lines[pos][0] == 'jnz':
        lines[pos][0] = 'cpy'
      else:
        lines[pos][0] = 'jnz'
    
  while 0 <= i < len(lines) and len(ret) < 50:
    inst = lines[i]
    i += 1
    op, x, y = inst[0], inst[1], inst[-1]
    if op == 'cpy' and y in regs:
      regs[y] = regs[x] if x in regs else x
    elif op == 'inc':
      regs[x] += 1
    elif op == 'dec':
      regs[x] -= 1
    elif op == 'jnz':
      check = regs[x] if x in regs else x
      move = regs[y] if y in regs else y
      i += move - 1 if check != 0 else 0
    elif op == 'tgl':
      pos = regs[x] if x in regs else x
      exec_tgl(i + pos - 1)
    elif op == 'out':
      out = regs[x] if x in regs else x
      ret += str(out)

  return ret

def check(s):
  for i, _ in enumerate(s):
    if i == 0 or i == len(s) -1: continue
    if not s[i] != s[i-1] or not s[i] != s[i+1]:
      return False
  
  return True

def d25p1(txt):
  lines = [[x if x.isalpha() else int(x)
            for x in line.split()] for line in txt.splitlines()]
  for val in range(200):
    regs = {
      'a': val,
      'b': 0,
      'c': 0,
      'd': 0
    }
    ret = decrypt(regs, lines)
    if check(ret):
      return val, ret

# It's day 25
day = 25
txt = get_input(day)
val, ret = d25p1(txt)
print(val, ret)

175 01010101010101010101010101010101010101010101010101
time: 13 s (started: 2021-03-16 20:46:27 +00:00)


In [ ]:
#@title Part 2 { form-width: "20%" }
# If you reading this, MERRY XMAS <3